<a href="https://colab.research.google.com/github/pavansai26/FastAPI-microservice-to-classify-the-intents/blob/main/FastAPI_microservice_to_classify_the_intents_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

how to create a FastAPI microservice that uses the Universal Sentence Encoder model to classify the intent of input sentences

First, you need to install the fastapi, uvicorn, and pydantic libraries using pip:



In [1]:
!pip install fastapi uvicorn pydantic pymongo


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.1/492.1 KB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 KB 27.8 MB/s eta 0:00:00


Then, you can create a file called app.py with the following code:



In [3]:
import tensorflow_hub as hub
import tensorflow as tf
from fastapi import FastAPI
from pydantic import BaseModel
import pymongo

In [ ]:
# Define the FastAPI app
app = FastAPI()

# Define the input data model
class InputData(BaseModel):
    text: str

# Define the output data model
class OutputData(BaseModel):
    intent: str

In [ ]:
# Load the Universal Sentence Encoder model
model_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
model = hub.load(model_url)

In [ ]:
# Define the intent classes
intent_classes = ['greeting', 'goodbye', 'question', 'statement']

In [ ]:
# Connect to the MongoDB database
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["intent_data"]
intents_collection = db["intents"]

In [ ]:
# Define a function to classify a sentence based on its embedding
def classify_intent(sentence_embedding):
    # Define the classifier model
    classifier_model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(512,)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(len(intent_classes), activation='softmax')
    ])
    classifier_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
  # Retrieve the intent data from the MongoDB database
    intents = intents_collection.find()
    sentences = []
    intent_labels = []
    for intent in intents:
        sentences.append(intent['text'])
        intent_label = intent_classes.index(intent['intent'])
        intent_labels.append(intent_label)

In [ ]:
  # Encode the sentences using the Universal Sentence Encoder model
    sentence_embeddings = model(sentences)

    # Train the classifier model on the sentence embeddings and intent labels
    intent_labels = tf.keras.utils.to_categorical(intent_labels)
    classifier_model.fit(sentence_embeddings, intent_labels, epochs=50, verbose=0)

    # Use the classifier model to predict the intent of the input sentence
    intent_probabilities = classifier_model.predict(tf.reshape(sentence_embedding, (1, 512)))
    predicted_intent_index = tf.argmax(intent_probabilities, axis=1).numpy()[0]
    predicted_intent = intent_classes[predicted_intent_index]
    return predicted_intent

In [ ]:
# Define a route to classify input sentences
@app.post("/classify_intent", response_model=OutputData)
def classify(input_data: InputData):
    sentence_embedding = model([input_data.text])
    predicted_intent = classify_intent(sentence_embedding)
    return {"intent": predicted_intent}


The above code defines a FastAPI app, a data model for the input and output data, loads the Universal Sentence Encoder model, connects to a MongoDB database, defines a function to classify the intent of input sentences using the database-stored intent data, and defines a route to classify input sentences and return the predicted intent.

To run the microservice, you can use the following command:



In [ ]:
uvicorn app:app --reload


This command starts the FastAPI app and reloads the server automatically whenever changes are made to the code. You can then send POST requests to the http://localhost:8000/classify_intent endpoint with a JSON payload containing a "text" field with the input sentence to classify